**If you find this notebook useful, please give an upvote xD**<br>
**Thanks!**

# Weighted Mean Baseline

**Goal:** Create a simple baseline using a constant prediction for all test sample. This will come from the mean value of each target variable scaled by proposed sample weights: 

- 1 for all healthy labels.
- 2 for low grade solid organ injuries (liver, spleen, kidney).
- 4 for high grade solid organ injuries.
- 2 for bowel injuries.
- 6 for extravasation.
- 6 for the auto-generated any_injury label.

In addition, we provide a method to evaluate the score for the training data and investigate alternative scale factors. This will be used to highlight the challenges of unbalanced data and weighted scoring metrics. 

# Imports

In [1]:
import numpy as np
import pandas as pd
import pandas.api.types
import sklearn.metrics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load Data

In [2]:
# Only requires the training target data. 
y_train = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')

y_train.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [3]:
# List of Targets
Injuries = ['bowel_healthy', 'bowel_injury', 
            'extravasation_healthy', 'extravasation_injury', 
            'kidney_healthy', 'kidney_low', 'kidney_high', 
            'liver_healthy', 'liver_low', 'liver_high', 
            'spleen_healthy', 'spleen_low', 'spleen_high', 
            'any_injury']

# Target EDA

In [4]:
y_train[Injuries].describe()

,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
count,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000
mean,0.979663,0.020337,0.936447,0.063553,0.942167,0.036543,0.021290,0.897998,0.082301,0.019701,0.887512,0.063235,0.049253,0.271687
std,0.141172,0.141172,0.243993,0.243993,0.233464,0.187666,0.144373,0.302699,0.274866,0.138994,0.316016,0.243424,0.216431,0.444900
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# [Score](https://www.kaggle.com/code/metric/rsna-trauma-metric/notebook)

In [5]:
# I'm not sure if this is needed!
# class ParticipantVisibleError(Exception):
#     pass

def normalize_probabilities_to_one(df: pd.DataFrame, group_columns: list) -> pd.DataFrame:
    # Normalize the sum of each row's probabilities to 100%.
    # 0.75, 0.75 => 0.5, 0.5
    # 0.1, 0.1 => 0.5, 0.5
    row_totals = df[group_columns].sum(axis=1)
    if row_totals.min() == 0:
        raise ParticipantVisibleError('All rows must contain at least one non-zero prediction')
    for col in group_columns:
        df[col] /= row_totals
    return df


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Pseudocode:
    1. For every label group (liver, bowel, etc):
        - Normalize the sum of each row's probabilities to 100%.
        - Calculate the sample weighted log loss.
    2. Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    3. Calculate the sample weighted log loss for the new label group
    4. Return the average of all of the label group log losses as the final score.
    '''
    del solution[row_id_column_name]
    del submission[row_id_column_name]

    # Run basic QC checks on the inputs
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission.values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution.min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission.min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    # Calculate the label group log losses
    binary_targets = ['bowel', 'extravasation']
    triple_level_targets = ['kidney', 'liver', 'spleen']
    all_target_categories = binary_targets + triple_level_targets

    label_group_losses = []
    for category in all_target_categories:
        if category in binary_targets:
            col_group = [f'{category}_healthy', f'{category}_injury']
        else:
            col_group = [f'{category}_healthy', f'{category}_low', f'{category}_high']

        solution = normalize_probabilities_to_one(solution, col_group)

        for col in col_group:
            if col not in submission.columns:
                raise ParticipantVisibleError(f'Missing submission column {col}')
        submission = normalize_probabilities_to_one(submission, col_group)
        label_group_losses.append(
            sklearn.metrics.log_loss(
                y_true=solution[col_group].values,
                y_pred=submission[col_group].values,
                sample_weight=solution[f'{category}_weight'].values
            )
        )

    # Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    healthy_cols = [x + '_healthy' for x in all_target_categories]
    any_injury_labels = (1 - solution[healthy_cols]).max(axis=1)
    any_injury_predictions = (1 - submission[healthy_cols]).max(axis=1)
    any_injury_loss = sklearn.metrics.log_loss(
        y_true=any_injury_labels.values,
        y_pred=any_injury_predictions.values,
        sample_weight=solution['any_injury_weight'].values
    )

    label_group_losses.append(any_injury_loss)
    return np.mean(label_group_losses)

In order to evaluate the score, the appropriate weights need to be assigned. The score function defined above expects sample weights for each category of injury (bowel, extravasation, kidney, liver, spleen, any). The sample weight is assigned based on the true target for a given category. For example, if a sample has a low grade kidney injury, then 

    [kidney_healthy, kidney_low, kidney_high] = [0,1,0]

and we would set kidney_weight = 2 for for that sample.

In [6]:
# Assign the appropriate weights to each category
def create_training_solution(y_train):
    sol_train = y_train.copy()
    
    # bowel healthy|injury sample weight = 1|2
    sol_train['bowel_weight'] = np.where(sol_train['bowel_injury'] == 1, 2, 1)
    
    # extravasation healthy/injury sample weight = 1|6
    sol_train['extravasation_weight'] = np.where(sol_train['extravasation_injury'] == 1, 6, 1)
    
    # kidney healthy|low|high sample weight = 1|2|4
    sol_train['kidney_weight'] = np.where(sol_train['kidney_low'] == 1, 2, np.where(sol_train['kidney_high'] == 1, 4, 1))
    
    # liver healthy|low|high sample weight = 1|2|4
    sol_train['liver_weight'] = np.where(sol_train['liver_low'] == 1, 2, np.where(sol_train['liver_high'] == 1, 4, 1))
    
    # spleen healthy|low|high sample weight = 1|2|4
    sol_train['spleen_weight'] = np.where(sol_train['spleen_low'] == 1, 2, np.where(sol_train['spleen_high'] == 1, 4, 1))
    
    # any healthy|injury sample weight = 1|6
    sol_train['any_injury_weight'] = np.where(sol_train['any_injury'] == 1, 6, 1)
    return sol_train

In [7]:
solution_train = create_training_solution(y_train)

# predict a constant using the mean of the training data
y_pred = y_train.copy()
y_pred[Injuries] = y_train[Injuries].mean().tolist()

no_scale_score = score(solution_train,y_pred,'patient_id')
print(f'Training score without scaling: {no_scale_score}')

Training score without scaling: 0.7860663285561644


Because the score weighs each target differently, we expect that the score can be improved by scaling by each target by their respective weight. 

In [8]:
# Group by different sample weights
scale_by_2 = ['liver_low','spleen_low','kidney_low','spleen_high'] # 
scale_by_4 = ['bowel_injury', 'kidney_high', 'liver_high'] # 
scale_by_6 = ['extravasation_injury'] # ,'any_injury'
scale_healthy = ['kidney_healthy', 'bowel_healthy'] # ', 'extravasation_healthy','liver_healthy', 'spleen_healthy'

# Scale factors based on described metric 

MULTIPLIER = 0.9859079

# MULTIPLIER NEXT = 0.0.995908001
# MULTIPLIER BEST = 0.995908
# MULTIPLIER DONE = 0.985908, 0.99590805, 0.995908001, 0.9959080000000001, 0.995907999999999, 0.99590799, 0.99590801, 0.9959079, 0.9959081, 0.9959082, 0.9959087, 0.995906, 0.995907, 0.995909, 0.9959085, 0.995913, 0.995905, 0.995915, 0.99999999, 0.98989999, 0.978645131, 0.998856151, 0.999153158, 1.015196, 1.005318, 0.97453, 0.987543, 0.99591

sf_2 = 2.753135 * MULTIPLIER
sf_4 = 6.7531358 * MULTIPLIER
sf_6 = 28.78613121 * MULTIPLIER
scale_h = 0.995215 * MULTIPLIER


# The score function deletes the ID column so we remake it
solution_train = create_training_solution(y_train)

# Reset the prediction
y_pred = y_train.copy()
y_pred[Injuries] = y_train[Injuries].mean().tolist()

# Scale each target 
y_pred[scale_by_2] *=sf_2
y_pred[scale_by_4] *=sf_4
y_pred[scale_by_6] *=sf_6
y_pred[scale_healthy] *=scale_h

weight_scale_score = score(solution_train,y_pred,'patient_id')
print(f'Training score with weight scaling: {weight_scale_score}')

Training score with weight scaling: 0.6134849721304003


We can do even better by increasing the scale factors. We will investigate one alternative option. The goal is to highlight that the most accurate prediction doesn't necessarily give the highest score!

In [9]:
# Update scale factors to improve score
# sf_2 = 2
# sf_4 = 4
# sf_6 = 14

# The score function deletes the ID column so we remake it
solution_train = create_training_solution(y_train)

# Reset the prediction, again
y_pred = y_train.copy()
y_pred[Injuries] = y_train[Injuries].mean().tolist()

# Scale each target 
y_pred[scale_by_2] *=sf_2
y_pred[scale_by_4] *=sf_4
y_pred[scale_by_6] *=sf_6
y_pred[scale_healthy] *=scale_h

improved_scale_score = score(solution_train,y_pred,'patient_id')
print(f'Training score with better scaling: {improved_scale_score}')

Training score with better scaling: 0.6134849721304003


We will stick with this choice of scale factors as it appears to be better. Note, this appears to be near the optimal choice of scale factors for the mean, it is not necessarily the optimal choice of constant solution even for the training data. The optimal choice may given equal scaling to each injury which should have the same weight!

When producing proper predictions, a constant scale factor may not be the optimal choice!

# Submission

In [10]:
# Load submission template 
submission = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv')

# Set output to mean of training data
submission[Injuries] = y_train[Injuries].mean().tolist()

# Scale each category by desired scale factor
submission[scale_by_2] *=sf_2
submission[scale_by_4] *=sf_4
submission[scale_by_6] *=sf_6
submission[scale_healthy] *=scale_h

# Save Submission!
submission.to_csv('submission.csv', index=False)

In [11]:
submission

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,48843,0.961236,0.135402,0.936447,1.803653,0.924445,0.099189,0.141749,0.897998,0.223392,0.131171,0.887512,0.171641,0.13369,0.271687
1,50046,0.961236,0.135402,0.936447,1.803653,0.924445,0.099189,0.141749,0.897998,0.223392,0.131171,0.887512,0.171641,0.13369,0.271687
2,63706,0.961236,0.135402,0.936447,1.803653,0.924445,0.099189,0.141749,0.897998,0.223392,0.131171,0.887512,0.171641,0.13369,0.271687
